# Feature Extraction

## Reading the input

In [2]:
import numpy as np
import pyedflib
import pandas
import pyeeg
from pandas import DataFrame
from matplotlib import pyplot as plt
from nitime import utils
from nitime import algorithms as alg
from nitime.timeseries import TimeSeries
from nitime.viz import plot_tseries
import csv
import pywt
from numpy import zeros, floor, log10, log, mean, array, sqrt, vstack, cumsum, ones, log2, std
import scipy.stats as sp
from spectrum import *
import numpy.fft
from numpy.fft import fft
from numpy.linalg import svd, lstsq
from os import listdir
from os.path import isfile, join
import mne.io

/home/marina/anaconda/lib/python3.7/site-packages/mpl_toolkits/axes_grid/__init__.py:12: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  obj_type='module')


#### Names of all features

In [3]:
names = ['Activity','Mobility','Complexity','Kurtosis','2nd Difference Mean','2nd Difference Max','Coeffiecient of Variation','Skewness','1st Difference Mean','1st Difference Max',
          'Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Energy',
          'Wavelet Approximate Entropy','Wavelet Detailed Entropy','Variance','Mean of Vertex to Vertex Slope','FFT Delta MaxPower','FFT Theta MaxPower','FFT Alpha MaxPower','FFT Beta MaxPower',
          'Autro Regressive Mode Order 3 Coefficients for each channel ->']
channel = ['ch1','ch2','ch3','ch4','ch5','ch6','ch7','ch8','ch9','ch10','ch11','ch12','ch13','ch14','ch15']


# Hjorth Parameters    

In [4]:
def first_order_diff(X):
    D = []
    for i in range(1,len(X)):
        D.append(X[i]-X[i-1])

    return D

In [5]:
def hjorth2(X, D = None):

    
    if D is None:
        D = first_order_diff(X)
    
    D.insert(0, X[0]) # pad the first difference
    D = array(D)
    n = len(X)
    M2 = float(sum(D ** 2)) / n
    TP = sum(array(X) ** 2)
    M4 = 0;
    for i in range(1, len(D)):
        M4 += (D[i] - D[i - 1]) ** 2
        M4 = M4 / n

    return sqrt(M2 / TP), sqrt(float(M4) * TP / M2 / M2)

In [6]:
def bin_power(X,Band,Fs):


    C = fft(X)
    C = abs(C)
    Power =zeros(len(Band))
    for Freq_Index in range(0,len(Band)-1):
        Freq = Band[Freq_Index]
        Next_Freq = Band[Freq_Index+1]
        Power[Freq_Index] = sum(C[int(floor(Freq/Fs*len(X))):int(floor(Next_Freq/Fs*len(X)))])
    Power_Ratio = Power/sum(Power)
    return Power, Power_Ratio

# Kurtosis , 2nd Diff Mean, 2nd Diff Max

In [7]:
def kurtosis(a):
    b = a # Extracting the data from the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    k = 0; # For counting the current row no.
    for i in b:
        mean_i = np.mean(i) # Saving the mean of array i
        std_i = np.std(i) # Saving the standard deviation of array i
        t = 0.0
        for j in i:
            t += (pow((j-mean_i)/std_i,4)-3)
        kurtosis_i = t/len(i) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
        output[k] = kurtosis_i # Saving the kurtosis in the array created
        k +=1 # Updating the current row no.
    return np.sum(output)/14

##----------------------------------------- End Kurtosis Function ----------------------------##


##------------------------------------- Begin 2ndDiffMean(Absolute difference) Function ------##
##-------------------------- [ Input: 2D array (row: Channels, column: Data)] --------------- ##
##-------------------  -- [ Output: 1D array (2ndDiffMean values for each channel)] ----------##

def secDiffMean(a):
    b = a # Extracting the data of the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    for i in b:
        t = 0.0
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        for j in range(len(i)-2):
            t += abs(temp1[j+1]-temp1[j]) # Summing the 2nd Diffs
        output[k] = t/(len(i)-2) # Calculating the mean of the 2nd Diffs
        k +=1 # Updating the current row no.
    return np.sum(output)/14

##------------------------------------- End 2ndDiffMean Function----- -------------------------##


##------------------------------------- Begin 2ndDiffMax Function(Absolute difference) --------##
##-------------------------- [ Input: 2D array (row: Channels, column: Data)] -----------------##
##--------------------- [ Output: 1D array (2ndDiffMax values for each channel)] --------------##

def secDiffMax(a):
    b = a # Extracting the data from the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    t = 0.0
    for i in b:
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            
        t = temp1[1] - temp1[0]
        for j in range(len(i)-2):
            if abs(temp1[j+1]-temp1[j]) > t :
            	t = temp1[j+1]-temp1[j] # Comparing current Diff with the last updated Diff Max

        output[k] = t # Storing the 2nd Diff Max for channel k
        k +=1 # Updating the current row no.
    return np.sum(output)/14



# Coefficient of Varaition

In [8]:
def coeff_var(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for i in b:
        mean_i = np.mean(i) #Saving the mean of array i
        std_i = np.std(i) #Saving the standard deviation of array i
        output[k] = std_i/mean_i #computing coefficient of variation
        k=k+1
    return np.sum(output)/14

# Skewness , 1st Difference Mean, 1st Difference Max

In [9]:
def skewness(arr):
    data = arr 
    skew_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0 #current cell position in the output array
   
    for i in data:
        skew_array[index]=sp.stats.skew(i,axis=0,bias=True)
        index+=1 #updating the cell position
    return np.sum(skew_array)/14


def first_diff_mean(arr):
    data = arr 
    diff_mean_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0 #current cell position in the output array
   
    for i in data:
        sum=0.0#initializing the sum at the start of each iteration
        for j in range(len(i)-1):
            sum += abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
           
        diff_mean_array[index]=sum/(len(i)-1)
        index+=1 #updating the cell position
    return np.sum(diff_mean_array)/14


def first_diff_max(arr):
    data = arr 
    diff_max_array = np.zeros(len(data)) #Initialinling the array as all 0s
    first_diff = np.zeros(len(data[0])-1)#Initialinling the array as all 0s 
    index = 0 #current cell position in the output array
   
    for i in data:
        max=0.0#initializing at the start of each iteration
        for j in range(len(i)-1):
            first_diff[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            if first_diff[j]>max: 
                max=first_diff[j] # finding the maximum of the first differences
        diff_max_array[index]=max
        index+=1 #updating the cell position
    return np.sum(diff_max_array)/14



# Wavelet transform features

In [10]:
#Adapted Wavelet transform
def wavelet_features(epoch):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    #For each channel I compute the discret wavelet transform
    for i in range(14):
        cA,cD=pywt.dwt(epoch[i][:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    #I make the mean for each channel
    for x in range(14):   
        cA_mean.append(np.mean(cA_values[x]))
        cA_std.append(np.std(cA_values[x]))
        cA_Energy.append(np.sum(np.square(cA_values[x])))
        cD_mean.append(np.mean(cD_values[x]))		# mean and standard deviation values of coefficents of each channel is stored .
        cD_std.append(np.std(cD_values[x]))
        cD_Energy.append(np.sum(np.square(cD_values[x])))
        Entropy_D.append(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x]))))
        Entropy_A.append(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x]))))
    return np.sum(cA_mean)/14,np.sum(cA_std)/14,np.sum(cD_mean)/14,np.sum(cD_std)/14,np.sum(cA_Energy)/14,np.sum(cD_Energy)/14,np.sum(Entropy_A)/14,np.sum(Entropy_D)/14

## Variance and Mean of Vertex to Vertex Slope

In [11]:
import heapq

from scipy.signal import argrelextrema

def first_diff(i):
    b=i
    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c

#first_diff(s)

def slope_mean(p):
    b = np.array(p) #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]
        t_max = argrelextrema(x, np.greater)[0]
        amp_min = i[argrelextrema(x, np.less)[0]]
        t_min = argrelextrema(x, np.less)[0]
        t = np.concatenate((t_max,t_min),axis=0)
        t.sort()#sort on the basis of time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q]         
        output[k] = np.mean(res) 
        k=k+1
    return np.sum(output)/14






def slope_var(p):
    b = np.array(p) #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]#storing maxima value
        t_max = argrelextrema(x, np.greater)[0]#storing time for maxima
        amp_min = i[argrelextrema(x, np.less)[0]]#storing minima value
        t_min = argrelextrema(x, np.less)[0]#storing time for minima value
        t = np.concatenate((t_max,t_min),axis=0) #making a single matrix of all matrix
        t.sort() #sorting according to time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q] #calculating slope        
    
        output[k] = np.var(res) 
        k=k+1#counting k
    return np.sum(output)/14



## FFT features(Max Power)

In [12]:
from scipy import signal

def maxPwelch(data_win,Fs):
 
    
    BandF = [0.1, 3, 7, 12, 30]
    PMax = np.zeros([14,(len(BandF)-1)])
    
    for j in range(14):
        f,Psd = signal.welch(data_win[j], Fs)
        
        for i in range(len(BandF)-1):
            fr = np.where((f>BandF[i]) & (f<=BandF[i+1]))
            PMax[j,i] = np.max(Psd[fr])
    
    return np.sum(PMax[:,0])/14,np.sum(PMax[:,1])/14,np.sum(PMax[:,2])/14,np.sum(PMax[:,3])/14


## Shanon Entropy and Entropy Spectral

In [13]:
def spectral_entropy(Power_Ratio):

    Spectral_Entropy = 0
    for i in range(0, len(Power_Ratio) - 1):
        Spectral_Entropy += Power_Ratio[i] * log(Power_Ratio[i])
    Spectral_Entropy /= log(len(Power_Ratio))	# to save time, minus one is omitted
    return -1 * Spectral_Entropy

In [15]:
def in_range(Template, Scroll, Distance):


	for i in range(0,  len(Template)):
			if abs(Template[i] - Scroll[i]) > Distance:
			     return False
	return True


## Autoregressive model

### Autoregression model- Yule Walker Algorithm

In [16]:
def autogressiveModelParameters(epoch):
    feature = []
    #Order 11 for the autoregressive model, why?
    for i in range(14):
        coeff, sig = alg.AR_est_YW(np.array(epoch[i]), 11,)
        feature.append(coeff)
    
     
    return feature

### Autoregression model- Burg Algorithm

In [17]:
def autogressiveModelParametersBurg(labels):
    feature = []
    feature1 = []
    model_order = 3
    for i in range(14):
        AR, rho, ref = arburg(labels[i], model_order)
        feature.append(AR);
    for j in range(14):
        for i in range(model_order):
            feature1.append(feature[j][i])

    return feature1

## Hurst exponent


In [18]:
def hurst(X):

	N = len(X)
    
	T = array([float(i) for i in range(1,N+1)])
	Y = cumsum(X)
	Ave_T = Y/T
	
	S_T = zeros((N))
	R_T = zeros((N))
	for i in range(N):
		S_T[i] = std(X[:i+1])
		X_T = Y - T * Ave_T[i]
		R_T[i] = max(X_T[:i + 1]) - min(X_T[:i + 1])
    
	R_S = R_T / S_T
	R_S = log(R_S)
	n = log(T).reshape(N, 1)
	H = lstsq(n[1:], R_S[1:])[0]
	return H[0]


## Managing the data (1rst record), creating an array

In [19]:
file = pandas.read_csv("/home/marina/Documents/DTU/Advanced machine learning/Project/recordings_csv/"+"1_0.csv")
df =DataFrame(file)


#Selecting first row
#df.iloc[0,2]
#print(df['0'])
#df[1]
channels = []
for j in range(0,14):
    ch = []
    channels.append(ch)

#Creating an array of arrays with the data from each channel
for j in range(0,14):
    for i in range(1,23041):
        channels[j].append(df.iloc[j,i])
  
  

## Extracting the features per channel

In [91]:
#Extracting features
mobility = []
complexity = []
skewness_list = []
hurst_list = []

#Features that are computed per channel (each feature has 14 components, one per channel)
for j in range(0,14):
    mobility.append(hjorth2(channels[j])[0])
    complexity.append(hjorth2(channels[j])[1])
    skewness_list.append(skewness(channels[j]))
    #hurst_list.append(hurst(channels[j]))
    
autoregressive_burg =autogressiveModelParametersBurg(channels)
#Per cada canal en tinc 3
autoregressive= autogressiveModelParameters(channels)
 #Features that are not computed per channel, but per epoch

#Shannon entropy

   

In [73]:
Band = [0.1, 3, 7, 12, 30]
a = first_diff_mean(channels)
b = first_diff_max(channels)
kurt = kurtosis(channels)
secdiffmean = secDiffMean(channels)
secdiffmax = secDiffMax(channels)
coef_variation = coeff_var(channels)
slopemean = slope_mean(channels)
slopevar = slope_var(channels)
wavelet_features_list = (wavelet_features(channels))
w1 = wavelet_features_list[0]
w2 = wavelet_features_list[1]
w3 = wavelet_features_list[2]
w4 = wavelet_features_list[3]
w5 = wavelet_features_list[4]
w6 = wavelet_features_list[5]
w7 = wavelet_features_list[6]
w8 = wavelet_features_list[7]
#Per cada canal en tinc 3

maxPwelch_list = maxPwelch(channels,256)
welch1 = maxPwelch_list[0]
welch2 = maxPwelch_list[1]
welch3 = maxPwelch_list[2]
welch4 = maxPwelch_list[3]
binpower, binratio = bin_power(channels[0],Band, 256)
power1 = binpower[0]
power2 = binpower[1]
power3 = binpower[2]
power4 = binpower[3]
ratio1= binratio[0]
ratio2 = binratio[1]
ratio3 = binratio[2]
ratio4 = binratio[3]

spec_entropy = spectral_entropy(binratio)




/home/marina/anaconda/lib/python3.7/site-packages/nitime/utils.py:980: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  complex_result = (np.issubdtype(in1.dtype, np.complex) or
/home/marina/anaconda/lib/python3.7/site-packages/nitime/utils.py:981: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  np.issubdtype(in2.dtype, np.complex))
/home/marina/anaconda/lib/python3.7/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


## Storing the features into a csv (the ones that do not depend on the channel)

In [134]:
array_features = [a,b,kurt,secdiffmean,secdiffmax,coef_variation,slopemean,slopevar,w1,w2,w3,w4,w5,w6,w7,w8,welch1,welch2,welch3,welch4,power1,power2,power3,power4,ratio1,ratio2,ratio3,ratio4,spec_entropy]
name_features = ['first_diff_mean','first_diff_max','kurtosis','sec_diff_mean','sec_diff_max','coef_variation','slopemean','slopevar','w1','w2','w3','w4','w5','w6','w7','w8','welch1','welch2','welch3','welch4','power1','power2','power3','power4','ratio1','ratio2','ratio3','ratio4','spec_entropy']

df = DataFrame(array_features)

df.rename(index={0:'first_diff_mean',1:'first_diff_max',2:'kurtosis',3:'sec_diff_mean',4:'sec_diff_max',5:'coef_variation',6:'slopemean',7:'slopevar',8:'w1',9:'w2',10:'w3',11:'w4',12:'w5',13:'w6',14:'w7',15:'w8',16:'welch1',17:'welch2',18:'welch3',19:'welch4',20:'power1',21:'power2',22:'power3',23:'power4',24:'ratio1',25:'ratio2',26:'ratio3',27:'ratio4',28:'spec_entropy'}, inplace=True)
df.to_csv("features_per_recording_no_channel.csv")


## Storing the features per channel

In [133]:
data = np.array([mobility, complexity, skewness_list])
transpose = data.transpose()
df2 = DataFrame(transpose)
df2.rename(index={0:'ch1',1:'ch2',2:'ch3',3:'ch4',4:'ch5',5:'ch6',6:'ch7',7:'ch8',8:'ch9',9:'ch10',10:'ch11',11:'ch12',12:'ch13',13:'ch14'}, inplace=True)
df2.columns = ['Mobility','Complexity','skewness']
df2.to_csv("features_per_channel.csv")
